In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim

df = pd.read_csv("/content/drive/MyDrive/Scrapping/electricity_train.csv")



In [ ]:
df.head()


,timestamp,consumption
0,2011-01-01 00:15:00,-0.704319
1,2011-01-01 00:30:00,-0.704319
2,2011-01-01 00:45:00,-0.678983
3,2011-01-01 01:00:00,-0.653647
4,2011-01-01 01:15:00,-0.704319


In [ ]:
def create_sequences(df, seq_length):
    xs, ys = [], []
    # Iterate over data indices
    for i in range(len(df) - seq_length):
      	# Define inputs
        x = df.iloc[i:(i+seq_length), 1]
        # Define target
        y = df.iloc[i+seq_length, 1]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

In [ ]:
x_train, y_train = create_sequences(df, 24*4)
print(x_train.shape, y_train.shape)

(105119, 96) (105119,)


In [ ]:
from torch.utils.data import TensorDataset
from torch.utils.data import Dataset, DataLoader
batch_size = 32

dataset_train = TensorDataset(
    torch.from_numpy(x_train).float(),
    torch.from_numpy(y_train).float()
)

dataloader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)




## RNN

In [ ]:
class Net(nn.Module):
  def __init__(self):
     super().__init__()
     self.rnn = nn.RNN(
         input_size = 1,
         hidden_size = 32,
         num_layers = 2,
         batch_first = True,)
     self.fc = nn.Linear(32,1)

  def forward(self, x):
    ho = torch.zeros(2, x.size(0), 32)
    out,_ = self.rnn(x, ho)
    out = self.fc(out[:, -1,:])
    return out


In [ ]:
net = Net()
learning_rate = 0.0001
num_epochs = 10
criterion = nn.MSELoss()
optimizer = optim.Adam(net.parameters(), lr=learning_rate)


for epoch in range(num_epochs):
    for seq, label in dataloader_train:

        seq = seq.view(-1,96,  1)



        outputs = net(seq)


        loss = criterion(outputs.squeeze(), label)


        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


Epoch [1/10], Loss: 0.1455
Epoch [2/10], Loss: 0.0092
Epoch [3/10], Loss: 0.0915
Epoch [4/10], Loss: 0.0092
Epoch [5/10], Loss: 0.0258
Epoch [6/10], Loss: 0.0117
Epoch [7/10], Loss: 0.0169
Epoch [8/10], Loss: 0.0252
Epoch [9/10], Loss: 0.0672
Epoch [10/10], Loss: 0.0513


## Model Evatuation

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Scrapping/electricity_test.csv")

In [ ]:
x_test, y_test = create_sequences(df, 24*4)
print(x_train.shape, y_train.shape)

(105119, 96) (105119,)


In [ ]:
dataset_test = TensorDataset(
    torch.from_numpy(x_test).float(),
    torch.from_numpy(y_test).float()
)

dataloader_test = DataLoader(dataset_test, batch_size=batch_size)


In [ ]:
pip install torchmetrics


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 841.5/841.5 kB 5.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:
import torchmetrics
from torchmetrics import MeanSquaredError
mse = torchmetrics.MeanSquaredError()

net.eval()
with torch.no_grad():
    for seqs, labels in dataloader_test:
        seqs = seqs.view(-1, 96, 1)

        outputs = net(seqs).squeeze()
        mse(outputs, labels)


test_mse = mse.compute()
print(f"Test MSE: {test_mse}")

Test MSE: 0.04379766434431076


## LSTM

In [ ]:
class Net(nn.Module):
  def __init__(self):
     super().__init__()
     self.lstm = nn.LSTM(
         input_size = 1,
         hidden_size = 32,
         num_layers = 2,
         batch_first = True,)
     self.fc = nn.Linear(32,1)

  def forward(self, x):
    ho = torch.zeros(2, x.size(0), 32)
    co = torch.zeros(2, x.size(0), 32)
    out,_ = self.lstm(x, (ho,co))
    out = self.fc(out[:, -1,:])
    return out

## GRU

In [ ]:
class Net(nn.Module):
  def __init__(self):
     super().__init__()
     self.gru = nn.GRU(
         input_size = 1,
         hidden_size = 32,
         num_layers = 2,
         batch_first = True,)
     self.fc = nn.Linear(32,1)

  def forward(self, x):
    ho = torch.zeros(2, x.size(0), 32)
    out,_ = self.gru(x, ho)
    out = self.fc(out[:, -1,:])
    return out